We practiced web scraping when all the information is in a single table of a single page in a site. What happens when we want to scrape information from multiple pages?

## IMDB multi page scraping

Go to https://www.imdb.com/search/title/ and enter the following parameters, leaving all other fields blank or with its default value:

- Title Type: Feature film

- Release date: From 1990 to 1992

- User Rating: 7.5 to "-"

The page you get should be familiar. There's a list with movies and each movie has its title, release year, crew, etc. You could inspect the page and build the code to collect the date.

However, the results we obtained contain 627 movies, and each page only contains 50 of them (you can change the settings to obtain up to 250 movies/page, but that still won't make it till the end).

The way to automatize web scraping in these cases is to look at the URLs The one we've obtained is the following:

[https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,]

If you scroll down and click on "Next", the URL is now: https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt

Click again on "Next" and here's the new URL: https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt

The patterns are clear: our search options are in the parameters title_type, release_date and user_rating. Then, we have the start parameter, which jumps in intervals of 50, and the ref_ parameter, which takes the value of "adv_nxt".

Let's do some requests:

In [59]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests

In [60]:
# 2. url: we start with the 'second' page
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt"

In [61]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [62]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film,
Released between 1990-01-01 and 1992-12-31,
User Rating at least 7.5
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?title_type=feature&amp;release_date=1990-01-0

Now, we'll have to build a loop where we simply replace the 51 for all the other values (jumping by 50) up until the end of the results. For simplicity, we will build manually this list of values to iterate through:



In [63]:
iterations = range(1, 631, 50)

for i in iterations:
    start_at= str(i)
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=" + start_at + "&ref_=adv_nxt"
    print(url)
    print("\n")

https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=1&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=151&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=201&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=251&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=301&ref_=adv_nxt


https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-

In [64]:
# To make it more "human", we can randomize the waiting time:
from time import sleep
from random import randint

for i in range(5):
    print(i)
    wait_time = randint(1,4)
    print("I will sleep for " + str(wait_time) + " seconds.")
    sleep(wait_time)

0
I will sleep for 1 seconds.
1
I will sleep for 2 seconds.
2
I will sleep for 4 seconds.
3
I will sleep for 4 seconds.
4
I will sleep for 2 seconds.


We will now scrape all the pages and store the response into a list - waiting a few seconds in between requests:

In [65]:
pages = []

for i in iterations:

    # assemble the url:
    start_at= str(i)
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=" + start_at + "&ref_=adv_nxt"

    # download html with a get request:
    response = requests.get(url)

    # monitor the process by printing the status code
    print("Status code: " + str(response.status_code))

    # store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,3)
    print("I will sleep for " + str(wait_time) + " second/s.")
    sleep(wait_time)

Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 1 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 1 second/s.
Status code: 200
I will sleep for 1 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 1 second/s.
Status code: 200
I will sleep for 1 second/s.


Note how if you print the object pages after running the code above, you'll just see the response code messages, but the html code is still accessible and you can parse it the same way we've always done:

In [66]:
BeautifulSoup(pages[0].content, "html.parser")


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film,
Released between 1990-01-01 and 1992-12-31,
User Rating at least 7.5
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?title_type=feature&amp;release_date=1990-01-0

It's the moment to build the code that collects all the 631 movie titles and their synopsis in a dataframe.

#### titles

In [67]:
# Parse just the first page, for testing purposes
soup = BeautifulSoup(pages[0].content, "html.parser")


# Paste the Selector from the first movie title copied from Chrome Dev Tools
soup.select("#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > h3 > a")

# Trim the selection: now it grabs all the titles
soup.select(".lister-item-content   h3  a")

soup.select(".lister-item-header a")

[<a href="/title/tt0099685/">GoodFellas - Drei Jahrzehnte in der Mafia</a>,
 <a href="/title/tt0102926/">Das Schweigen der Lämmer</a>,
 <a href="/title/tt0099348/">Der mit dem Wolf tanzt</a>,
 <a href="/title/tt0103064/">Terminator 2: Tag der Abrechnung</a>,
 <a href="/title/tt0105236/">Reservoir Dogs: Wilde Hunde</a>,
 <a href="/title/tt0104952/">Mein Vetter Winnie</a>,
 <a href="/title/tt0099785/">Kevin - Allein zu Haus</a>,
 <a href="/title/tt0099674/">Der Pate 3</a>,
 <a href="/title/tt0102138/">JFK: Tatort Dallas</a>,
 <a href="/title/tt0104797/">Malcolm X</a>,
 <a href="/title/tt0100802/">Total Recall - Die totale Erinnerung</a>,
 <a href="/title/tt0104257/">Eine Frage der Ehre</a>,
 <a href="/title/tt0105695/">Erbarmungslos</a>,
 <a href="/title/tt0101414/">Die Schöne und das Biest</a>,
 <a href="/title/tt0104691/">Der letzte Mohikaner</a>,
 <a href="/title/tt0105323/">Der Duft der Frauen</a>,
 <a href="/title/tt0103639/">Aladdin</a>,
 <a href="/title/tt0103074/">Thelma &amp; Lo

#### synopsis

In [68]:
# Paste the Selector from the first movie title copied from Chrome Dev Tools
soup.select("#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > p:nth-child(4)")

[<p class="text-muted">
 The story of <a href="/name/nm1453737">Henry Hill</a> and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.</p>]

In [69]:
# Trim the selection: now it grabs all the titles
soup.select("div.lister-item-content > p:nth-child(4)")

[<p class="text-muted">
 The story of <a href="/name/nm1453737">Henry Hill</a> and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.</p>,
 <p class="text-muted">
 A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.</p>,
 <p class="text-muted">
 Lieutenant John Dunbar, assigned to a remote western Civil War outpost, befriends wolves and Native Americans, making him an intolerable aberration in the military.</p>,
 <p class="text-muted">
 A cyborg, identical to the one who failed to kill Sarah Connor, must now protect her 10-year old son John from an even more advanced and powerful cyborg.</p>,
 <p class="text-muted">
 When a simple jewelry heist goes horribly wrong, the surviving criminals begin to suspect that one of them is a police informant.</p>,
 <p class="text

There are many approaches to do this. The one we'll follow is: 

- Loop through the pages we collected, parse them ("create the soup") and store the parsed pages in a list. 

- For each parsed page, select the "blocks of HTML elements" that contain all the information of each movie (the title, the synopsis and other stuff). 

- For each one of the "blocks" we collected in the previous step: 

    - Get the movie titles and store them in a list 

    - Get the synopsis and store them in a list

In [70]:
pages_parsed = []
titles = []
synopsis = []

for i in range(len(pages)):
    # parse all pages
    pages_parsed.append(BeautifulSoup(pages[i].content, "html.parser"))
    # select only the info about the movies
    movies_html = pages_parsed[i].select("div.lister-item-content")
    # for movie, store titles and reviews into lists
    for j in range(len(movies_html)):
        titles.append(movies_html[j].select("h3 > a")[0].get_text())
        synopsis.append(movies_html[j].select("p:nth-child(4)")[0].get_text().strip())


In [71]:
# Checking our output:
print(len(titles)) # output: 614
print(len(synopsis))  # output: 614

# Note: in your output the movie titles might be in English:
print(titles[0:3] )
print(synopsis[0:3]) 

650
650
['GoodFellas - Drei Jahrzehnte in der Mafia', 'Das Schweigen der Lämmer', 'Der mit dem Wolf tanzt']
['The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.', 'A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.', 'Lieutenant John Dunbar, assigned to a remote western Civil War outpost, befriends wolves and Native Americans, making him an intolerable aberration in the military.']


## Scraping presidents

Our objective is to create a dataframe with information about the presidents of the United States. To do this, we will go through this steps:

1. Scrape this [list of presidents of the United States](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States).


In [ ]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States"

# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
soup

In [81]:
presidents = []
presidents=soup.select(".wikitable tbody td b a")
soup.select(".wikitable > tbody:nth-child(3) > tr:nth-child(2) > td:nth-child(3) > b:nth-child(1)")

[]

In [82]:
presidents

[<a href="/wiki/President_of_the_United_States" title="President of the United States">President of the United States</a>,
 <a href="/wiki/List_of_United_States_senators_from_Ohio" title="List of United States senators from Ohio">United States Senator (Class 3) from Ohio</a>,
 <a href="/wiki/Lieutenant_Governor_of_Ohio" title="Lieutenant Governor of Ohio">Lieutenant Governor of Ohio</a>,
 <a href="/wiki/Republican_Party_(United_States)" title="Republican Party (United States)">Republican</a>,
 <a class="mw-redirect" href="/wiki/Governor_of_Ohio" title="Governor of Ohio">Governor of Ohio</a>,
 <a href="/wiki/Republican_Party_(United_States)" title="Republican Party (United States)">Republican</a>,
 <a href="/wiki/List_of_United_States_senators_from_Ohio" title="List of United States senators from Ohio">U.S. Senator</a>,
 <a href="/wiki/Ohio" title="Ohio">Ohio</a>,
 <a href="/wiki/Classes_of_United_States_senators" title="Classes of United States senators">Class 3</a>,
 <a href="/wiki/Jo

2. Collect all the links to the Wikipedia page of each president.


In [74]:
# we can access the links searching for the attribute "href"
# in each element
presidents[0]["href"]

'/wiki/George_Washington'

In [75]:
url = "https://en.wikipedia.org" + presidents[0]["href"]
url

'https://en.wikipedia.org/wiki/George_Washington'

In [76]:
# Now, we just assemble a new request to the link
# send request
url = "https://en.wikipedia.org/" + presidents[0]["href"]
response = requests.get(url)
response.status_code

# parse & store html
soup = BeautifulSoup(response.content, "html.parser")
soup.select(".infobox")

[<table class="infobox vcard"><tbody><tr><th class="infobox-above" colspan="2" style="font-size: 100%;"><div class="fn" style="font-size:125%;">George Washington</div></th></tr><tr><td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:Gilbert_Stuart_Williamstown_Portrait_of_George_Washington.jpg"><img alt="Head and shoulders portrait of George Washington" data-file-height="5615" data-file-width="4626" decoding="async" height="267" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Gilbert_Stuart_Williamstown_Portrait_of_George_Washington.jpg/220px-Gilbert_Stuart_Williamstown_Portrait_of_George_Washington.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Gilbert_Stuart_Williamstown_Portrait_of_George_Washington.jpg/330px-Gilbert_Stuart_Williamstown_Portrait_of_George_Washington.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Gilbert_Stuart_Williamstown_Portrait_of_George_Washington.jpg/440px-Gilbert_Stuart_Williamstown_Portrait_of_Geo

3. Scrape the Wikipedia page of each president.


In this step we could very well store the whole wikipedia page for each president, or just the tiny, final pieces of information. Storing the boxes is a middle ground (we don't have too much noise but retain the flexibility of deciding later which specific elements to extract).

When sending multiple requests, remember to be respectful by spacing the requests a few seconds from each other. We will also pring the success code to monitor that everything is going well:

In [77]:
# 2. find url and store it in a variable

presi_soups = []

for presi in presidents:
    # send request
    url = "https://en.wikipedia.org/" + presi["href"]
    response = requests.get(url)
    print(presi.get_text(), response.status_code)
    
    # parse & store html
    soup = BeautifulSoup(response.content, "html.parser")
    presi_soups.append(soup.select(".infobox"))
    
    # respectful nap:
    wait_time = randint(1,2)
    print("I will sleep for " + str(wait_time) + " second/s.")
    #sleep(wait_time)

George Washington 200
I will sleep for 1 second/s.
John Adams 200
I will sleep for 2 second/s.
Thomas Jefferson 200
I will sleep for 1 second/s.
James Madison 200
I will sleep for 1 second/s.
James Monroe 200
I will sleep for 2 second/s.
John Quincy Adams 200
I will sleep for 2 second/s.
Andrew Jackson 200
I will sleep for 1 second/s.
Martin Van Buren 200
I will sleep for 2 second/s.
William Henry Harrison 200
I will sleep for 1 second/s.
John Tyler 200
I will sleep for 1 second/s.
James K. Polk 200
I will sleep for 1 second/s.
Zachary Taylor 200
I will sleep for 2 second/s.
Millard Fillmore 200
I will sleep for 2 second/s.
Franklin Pierce 200
I will sleep for 1 second/s.
James Buchanan 200
I will sleep for 2 second/s.
Abraham Lincoln 200
I will sleep for 2 second/s.
Andrew Johnson 200
I will sleep for 2 second/s.
Ulysses S. Grant 200
I will sleep for 2 second/s.
Rutherford B. Hayes 200
I will sleep for 2 second/s.
James A. Garfield 200
I will sleep for 1 second/s.
Chester A. Arthur 20

KeyboardInterrupt: 

4. Find and store information about each president.


We extracted the 'infoboxes': now it's time to exctract especific pieces of information from them. Let's test what can we get from single presidents and then assemble a loop for all of them - as usual.

Here, we will use [the string argument](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#the-string-argument) in the find function, since wikipedia tags and classes are not always helpfulto locate. The string argument allows us to locate elements by its actual content.

In [ ]:
#Birthday
print(presi_soups[-1][0].select(".bday")[0].get_text())

#Political party
print(presi_soups[-1][0].find("th", string="Political party").parent.find("a").get_text())

#Number of sons/daughters
print(len(presi_soups[-1][0].find("th", string="Children").parent.find_all("li")))

In [ ]:
div.mb-4:nth-child(2) > div:nth-child(2) > h5:nth-child(1)

5. Organize the information in a dataframe where we have each president as a row and each variable we collected as a column. 
- you can take it from here ;) 

In [86]:
names = []
for i in range(0,46):
    

AttributeError: 'NoneType' object has no attribute 'get_text'

In [85]:
presidents

[<a href="/wiki/President_of_the_United_States" title="President of the United States">President of the United States</a>,
 <a href="/wiki/List_of_United_States_senators_from_Ohio" title="List of United States senators from Ohio">United States Senator (Class 3) from Ohio</a>,
 <a href="/wiki/Lieutenant_Governor_of_Ohio" title="Lieutenant Governor of Ohio">Lieutenant Governor of Ohio</a>,
 <a href="/wiki/Republican_Party_(United_States)" title="Republican Party (United States)">Republican</a>,
 <a class="mw-redirect" href="/wiki/Governor_of_Ohio" title="Governor of Ohio">Governor of Ohio</a>,
 <a href="/wiki/Republican_Party_(United_States)" title="Republican Party (United States)">Republican</a>,
 <a href="/wiki/List_of_United_States_senators_from_Ohio" title="List of United States senators from Ohio">U.S. Senator</a>,
 <a href="/wiki/Ohio" title="Ohio">Ohio</a>,
 <a href="/wiki/Classes_of_United_States_senators" title="Classes of United States senators">Class 3</a>,
 <a href="/wiki/Jo